## LIMPIEZA DE LA BASE DE DATOS PARA POWER BI 
Idea: Análisis de Estacionalidad de Streams en Spotify (2017–2021)

### 1. Carga y exploración inicial de datos

In [2]:
import pandas as pd
import numpy as np
ruta = r"C:/Users/clara/OneDrive - Universidade da Coruña/Escritorio/BDA/TAREFA_2_M7/POWER BI/data.csv"
df = pd.read_csv(ruta, sep="#")

df.info()
df.describe(include="all")
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321200 entries, 0 to 321199
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Position    321200 non-null  int64 
 1   Track Name  321182 non-null  object
 2   Artist      321182 non-null  object
 3   Streams     321200 non-null  int64 
 4   Date        321200 non-null  object
 5   Genre       321182 non-null  object
dtypes: int64(2), object(4)
memory usage: 14.7+ MB


,Position,Track Name,Artist,Streams,Date,Genre
0,1,Starboy,The Weeknd,3135625,2017-01-01,"['canadian pop', 'canadian contemporary r&b', ..."
1,2,Closer,The Chainsmokers,3015525,2017-01-01,"['pop', 'pop dance', 'tropical house', 'edm', ..."
2,3,Let Me Love You,DJ Snake,2545384,2017-01-01,"['pop', 'electronic trap', 'dance pop', 'edm',..."
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,2017-01-01,"['pop', 'uk dance', 'dance pop', 'uk funky', '..."
4,5,One Dance,Drake,2259887,2017-01-01,"['toronto rap', 'canadian pop', 'canadian hip ..."


### 2. Limpieza y normalización de la columna Genre

Los registros Genre viene como lista guardada en un string 
Usamos literal_eval para convertir a lista cuando sea necesario, y seleccionamos solo el primer género de la lista


In [3]:
import ast

df["Genre"] = df["Genre"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df["Genre"] = df["Genre"].apply(lambda x: x[0].strip().lower() if isinstance(x, list) and len(x) > 0 else None)
df[["Track Name", "Genre"]].head()
df.head()

,Position,Track Name,Artist,Streams,Date,Genre
0,1,Starboy,The Weeknd,3135625,2017-01-01,canadian pop
1,2,Closer,The Chainsmokers,3015525,2017-01-01,pop
2,3,Let Me Love You,DJ Snake,2545384,2017-01-01,pop
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,2017-01-01,pop
4,5,One Dance,Drake,2259887,2017-01-01,toronto rap


### 3. Transformación de la columna Date
- Convertimos fecha a formato datetime
- Extraemos el mes (número del 1 al 12)
- Extraemos el año (2017-2021)

In [4]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year
df.head()


,Position,Track Name,Artist,Streams,Date,Genre,Month,Year
0,1,Starboy,The Weeknd,3135625,2017-01-01,canadian pop,1,2017
1,2,Closer,The Chainsmokers,3015525,2017-01-01,pop,1,2017
2,3,Let Me Love You,DJ Snake,2545384,2017-01-01,pop,1,2017
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,2017-01-01,pop,1,2017
4,5,One Dance,Drake,2259887,2017-01-01,toronto rap,1,2017


### 4. Eliminación columnas innecesarias
- Date
- Position

In [5]:
df = df.drop(columns=["Date"])
df = df.drop(columns=["Position"])
df.head()

,Track Name,Artist,Streams,Genre,Month,Year
0,Starboy,The Weeknd,3135625,canadian pop,1,2017
1,Closer,The Chainsmokers,3015525,pop,1,2017
2,Let Me Love You,DJ Snake,2545384,pop,1,2017
3,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,pop,1,2017
4,One Dance,Drake,2259887,toronto rap,1,2017


### 5. Asegurar tipo numérico en Streams

In [6]:
df["Streams"] = pd.to_numeric(df["Streams"], errors="coerce")


### 6. Liempieza adicional de texto
- Quitamos espacios y pasamos a minúsculas
- Eliminamos géneros erróneos o confusos
- Eliminamos entradas no informativas

In [7]:
df["Artist"] = df["Artist"].astype(str).str.strip()
df["Genre"] = df["Genre"].astype(str).str.lower().str.strip()

mask_artist_digits = df["Artist"].str.match(r'^\d+$', na=False)
mask_artist_equals_year = df["Artist"].str.contains(r'^\s*202[0-9]\s*$', na=False)
mask_genero_numeric = df["Genre"].str.match(r'^\d+$', na=False)
mask_genero_short = df["Genre"].str.len() <= 2

df.loc[df["Genre"].str.contains("500|internal|server|error", na=False, case=False), "Genero"] = None

df.head()


,Track Name,Artist,Streams,Genre,Month,Year,Genero
0,Starboy,The Weeknd,3135625,canadian pop,1,2017,NaN
1,Closer,The Chainsmokers,3015525,pop,1,2017,NaN
2,Let Me Love You,DJ Snake,2545384,pop,1,2017,NaN
3,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,pop,1,2017,NaN
4,One Dance,Drake,2259887,toronto rap,1,2017,NaN


### 7. Normalización semántica de géneros


In [8]:
def normalizar_genero(texto):

    texto = texto.lower()

    if "pop" in texto:
        return "pop"
    
    if "trap" in texto:
        return "trap"

    if "rap" in texto:
        return "rap"

    if "latin" in texto or "reggaeton" in texto:
        return "latin"

    if "house" in texto:
        return "house"

    if "hip hop" in texto:
        return "hip hop"

    if "edm" in texto:
        return "edm"

    if "rock" in texto:
        return "rock"
    
    if "r&b" in texto:
        return "r&b"
    
    if "folk" in texto:
        return "folk"
    
    if "dance" in texto:
        return "dance"
    
    if "country" in texto:
        return "country"
    
    if "soul" in texto:
        return "soul"
    
    if "indie" in texto:
        return "indie"
    
    if "drill" in texto:
        return "drill"
    
    if "dutch" in texto:
        return "dutch"
    
    if "flamenco" in texto:
        return "flamenco"
    
    if "forro" in texto:
        return "forro"
    
    if "funk" in texto:
        return "funk"
    
    if "jazz" in texto:
        return "jazz"
    
    if "urban" in texto:
        return "urban"
    
    if "urbano" in texto:
        return "urban"

    return texto.strip()

df["Genero"] = df["Genre"].apply(normalizar_genero) # creamos nueva columna

# eliminar la columna de genero original
df = df.drop(columns=["Genre"])
df.head()

,Track Name,Artist,Streams,Month,Year,Genero
0,Starboy,The Weeknd,3135625,1,2017,pop
1,Closer,The Chainsmokers,3015525,1,2017,pop
2,Let Me Love You,DJ Snake,2545384,1,2017,pop
3,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,1,2017,pop
4,One Dance,Drake,2259887,1,2017,rap


### 8. Comporobación

In [10]:
df.columns

Index(['Track Name', 'Artist', 'Streams', 'Month', 'Year', 'Genero'], dtype='object')

### 9. Agregación mensual

In [11]:
df_monthly = df.groupby(["Track Name", "Artist", "Genero", "Year", "Month"], as_index=False)["Streams"].sum()

### 10. Exportar base de datos limpia

In [80]:
df.to_csv("spotify_mensual_limpio.csv", index=False, encoding="utf-8")
